In [16]:
from constants import DEV_BUCKET, STAGE_BUCKET, PROD_BUCKET, FOLDER_NAME
import tempfile
from dvc.api import DVCFileSystem
import s3fs
import yaml
import datetime

In [17]:
repo = 'git@github.com:ContextLogic/multitask-llm-rnd.git'
path = '/modelling/notebooks/convert_pl_to_hf_ckpt/content_moderation_v2/raw_ckpt'

In [18]:
MODEL_NAME = "content-moderation/v2"

In [19]:
with tempfile.TemporaryDirectory() as tmpdirname:
    print(tmpdirname)
    fs = DVCFileSystem(repo, subrepos=True)
    fs.get(path, tmpdirname, recursive=True)
    s3_file = s3fs.S3FileSystem()
    local_path = tmpdirname
    
    def rm_and_save(s3_path):
        if len(s3_file.ls(s3_path)) > 0:
            try:
                print(f'rm -rf {s3_path}')
                s3_file.rm(s3_path, recursive=True)
            except Exception as e:
                print(f'rm failed due to {e}')
        print(f'save to {s3_path}')
        s3_file.put(local_path, s3_path, recursive=True) 

    rm_and_save(f"{DEV_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}")
    rm_and_save(f"{STAGE_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}")
    rm_and_save(f"{PROD_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}")
    

/tmp/tmpc35r8zax
rm -rf structured-data-dev/coeus-gpu-multitask-ml/content-moderation/v2
save to structured-data-dev/coeus-gpu-multitask-ml/content-moderation/v2
rm -rf structured-data-stage/coeus-gpu-multitask-ml/content-moderation/v2
save to structured-data-stage/coeus-gpu-multitask-ml/content-moderation/v2
rm -rf structured-data-prod/coeus-gpu-multitask-ml/content-moderation/v2
save to structured-data-prod/coeus-gpu-multitask-ml/content-moderation/v2


In [20]:
yaml_dict = yaml.safe_load(open('models.yaml', 'r'))
if yaml_dict is None:
    yaml_dict = {}

In [21]:
assert MODEL_NAME.count('/') == 1

In [22]:
model_class, model_version = MODEL_NAME.split('/')

In [23]:
if model_class not in yaml_dict:
    yaml_dict[model_class] ={}
if model_version in yaml_dict[model_class]:
    print('overwrite previous config')
yaml_dict[model_class][model_version] = {
    "dvc": {
        "repo": repo, 
        "path": path, 
        "rev": "2c0a52930f90aee3e18828106b58ce219e45e82d"
    },
    "s3": {
        "dev": f"{DEV_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}",
        "stage": f"{STAGE_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}",
        "prod": f"{PROD_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}"
    },
    "metadata": {
        "library_type": "custom-python-func",
        "model_type": "clip-catboost",
        "yaml_update_time": str(datetime.datetime.now()),
        "max_batch_size": 20
    }
}

overwrite previous config


In [24]:
with open('models.yaml', 'w') as f:
    f.write(yaml.dump(yaml_dict))

In [25]:
import s3fs
fs = s3fs.S3FileSystem()
fs.get(f"{DEV_BUCKET}/{FOLDER_NAME}/{MODEL_NAME}", "tmp2", recursive=True)

[None, None, None, None, None]